In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("spark-test") \
    .master(os.getenv('SPARK_MASTER_URL')) \
    .config('spark.driver.cores', '1')\
    .config('spark.driver.memory', '1G')\
    .config('spark.executor.instances', '1')\
    .config('spark.executor.cores', '1')\
    .config('spark.executor.memory', '512m')\
    .config("spark.ui.port", "4045")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.791.jar") \
    .config("spark.hadoop.fs.defaultFS", "s3a://ozone-om") \
    .config("spark.hadoop.fs.s3a.access.key", "ozone-access-key") \
    .config("spark.hadoop.fs.s3a.secret.key", "ozone-secret-key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://ozone-s3g:9878") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.committer.name", "directory") \
    .config("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging") \
    .config("spark.hadoop.fs.s3a.multiobjectdelete.enable", "false") \
    .config("spark.hadoop.fs.s3a.directory.marker.retention", "keep") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "5000") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.socket.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.attempts.maximum", "50") \
    .config("spark.hadoop.fs.s3a.retry.limit", "50") \
    .config("spark.hadoop.fs.s3a.retry.interval", "5s") \
    .getOrCreate()

26/02/09 13:19:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.sql("SHOW DATABASES;").show()

26/02/08 11:37:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+



In [3]:
df = spark.createDataFrame([(1, "tgete", "2025-02-01"), (3, "vdsv", "2025-06-01")], ["id", "descr", "report_dt"])

In [4]:
df.show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [7]:
#insert into table
df.write\
    .mode("append") \
    .insertInto("test_db.test_table")

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: Unable to fetch table test_table. Invalid method name: 'get_table'

In [6]:
# write file
df.write\
    .mode("overwrite") \
    .format("parquet") \
    .save("s3a://mybucket/test_db/spark_test/test.parquet")

In [7]:
spark.read \
    .parquet("s3a://mybucket/test_db/spark_test/test.parquet") \
    .show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [8]:
spark.stop()